In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import LeaveOneOut, GridSearchCV, StratifiedKFold
from sklearn.feature_selection import mutual_info_classif, RFE, SelectKBest, f_classif
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, GaussianNoise
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.backend import clear_session
from sklearn.decomposition import PCA
import joblib
import warnings
warnings.filterwarnings("ignore")
seed = 1234
np.random.seed(seed)
from sklearn import svm, linear_model, ensemble, neighbors, tree, naive_bayes
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
# Importar librerías necesarias
import os
import joblib
from collections import Counter
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif, RFE
from sklearn.ensemble import RandomForestRegressor
import pickle
from collections import Counter


In [9]:
def split_data(X, y, split):
    X_train = X.iloc[split['X_train']]
    X_test = X.iloc[split['X_test']]
    y_train = y.iloc[split['X_train']]
    y_test = y.iloc[split['X_test']]
    #y_train = y.iloc[split['y_train']]
    #y_test = y.iloc[split['y_test']]
    #print(y_train)
    return X_train, X_test, y_train, y_test

In [10]:
def train_full_model(X, y_encoded,best_params, classifiers):
    # Preprocesamiento del conjunto de entrenamiento completo
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_final = X_scaled
    
    # Crear un directorio para guardar los modelos si no existe
    os.makedirs('modelos_finales_peru_bio', exist_ok=True)

    # Entrenar los modelos finales
    final_models = {}
    for name, clf_info in classifiers.items():
        model = clf_info['model']
        # Obtener los mejores hiperparámetros
        param_counts = Counter(tuple(sorted(params.items())) for params in best_params[name])
        most_common_params = dict(param_counts.most_common(1)[0][0])

        # Ajustar los nombres de los parámetros para que coincidan con el nombre del paso en el pipeline
        step_name = 'classifier'
        adjusted_params = {}
        for param_name, param_value in most_common_params.items():
            # Reemplazar el prefijo por el nombre del paso
            new_param_name = param_name.replace('classifier__', f'{step_name}__')
            adjusted_params[new_param_name] = param_value

        # Crear pipeline con el modelo (opcional, ya que el preprocesamiento ya se aplicó)
        pipeline = Pipeline([
            (step_name, model)
        ])

        # Establecer los hiperparámetros utilizando los nombres ajustados
        pipeline.set_params(**adjusted_params)
        # Entrenar el modelo con los datos preprocesados y seleccionados
        pipeline.fit(X_final, y_encoded)
        final_models[name] = pipeline
        # Guardar el modelo
        model_filename = f'modelos_finales_peru_bio/final_{name.lower().replace(" ", "_")}_model.pkl'
        joblib.dump(pipeline, model_filename)
        print(f'Modelo {name} guardado en {model_filename}')

In [11]:
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, balanced_accuracy_score, classification_report, confusion_matrix

def test_with_reserved(X_reserved, y_reserved,n_splits):
    scaler = StandardScaler()
    X_reserved_scaled = scaler.fit_transform(X_reserved)
    X_reserved_final = X_reserved_scaled
    # Cargar los modelos finales
    final_models = {}
    for model_filename in os.listdir('modelos_finales_peru_bio'):
        if model_filename.endswith('.pkl'):
            model_name = model_filename.split('_')[1]
            model = joblib.load(f'modelos_finales_peru_bio/{model_filename}')
            final_models[model_name] = model

    predictions_reserved = {}
    results_dir = 'resultados_peru_bio'
    file_name = os.path.join(results_dir, f'resultados_peru_bio_{n_splits+1}.txt')
    with open(file_name, 'a') as f:
        f.write(f"Resultados Split {n_splits+1}:\n")
        for name, model in final_models.items():
            # Realizar predicciones
            y_pred_reserved = model.predict(X_reserved_final)
            predictions_reserved[name] = y_pred_reserved
            
            # Calcular métricas
            acc_reserved = accuracy_score(y_reserved, y_pred_reserved)
            f1_reserved = f1_score(y_reserved, y_pred_reserved, average='weighted')
            mcc_reserved = matthews_corrcoef(y_reserved, y_pred_reserved)
            balanced_acc_reserved = balanced_accuracy_score(y_reserved, y_pred_reserved)
            conf_matrix = confusion_matrix(y_reserved, y_pred_reserved)
            
            #guarda los resultados por split en un archivo de texto
            f.write(f"Resultados para {name}:\n")
            f.write(f"  Accuracy: {acc_reserved:.4f}\n")
            f.write(f"  F1 Score (weighted): {f1_reserved:.4f}\n")
            f.write(f"  MCC: {mcc_reserved:.4f}\n")
            f.write(f"  Balanced Accuracy: {balanced_acc_reserved:.4f}\n")
            f.write("  Matriz de Confusión:\n")
            f.write(f"{conf_matrix}\n\n")

In [12]:
def evaluate_with_splits(X, X_reserved, y_encoded, y_reserved,n_split):
    classifiers = {
    'SVM Linear': {
        'model': svm.SVC(kernel='linear', class_weight='balanced'),
        'params': {'classifier__C': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]}
    },
    'SVM RBF': {
        'model': svm.SVC(kernel='rbf', class_weight='balanced'),
        'params': {
            'classifier__C': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
            'classifier__gamma': [0.0001, 0.001, 0.01, 0.1, 1, 'scale', 'auto']
        }
    },
    'Logistic Regression (LASSO)': {
        'model': linear_model.LogisticRegression(penalty='l1', solver='saga', max_iter=1000, class_weight='balanced'),
        'params': {
            'classifier__C': [0.01, 0.1, 1, 10]
        }
    },
    'Random Forest': {
        'model': ensemble.RandomForestClassifier(random_state=42, class_weight='balanced'),
        'params': {
            'classifier__n_estimators': [100, 200, 500],
            'classifier__max_depth': [None, 5, 10, 20],
            'classifier__min_samples_split': [2, 5, 10]
        }
    },
    'Boosting': {
        'model': ensemble.AdaBoostClassifier(random_state=42),
        'params': {
            'classifier__n_estimators': [100, 200, 500],
            'classifier__learning_rate': [0.01, 0.1, 1]
        }
    },
    'Decision Tree': {
        'model': tree.DecisionTreeClassifier(random_state=42, class_weight='balanced'),
        'params': {
            'classifier__max_depth': [None, 10, 20],
            'classifier__min_samples_split': [2, 5],
            'classifier__min_samples_leaf': [1, 2]
        }
    },
    'K-Nearest Neighbors': {
        'model': neighbors.KNeighborsClassifier(),
        'params': {
            'classifier__n_neighbors': [3, 5, 7],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__metric': ['euclidean', 'manhattan']
        }
    },
    'Naive Bayes': {
        'model': naive_bayes.GaussianNB(),
        'params': {
            # GaussianNB no tiene hiperparámetros principales para ajustar
        }
    },
    'MLP': {
        'model': MLPClassifier(max_iter=500, random_state=42),
        'params': {
            'classifier__hidden_layer_sizes': [(100,), (100, 50), (50, 25)],
            'classifier__activation': ['relu', 'tanh'],
            'classifier__learning_rate_init': [0.001, 0.01]
        }
    }
}
    # Inicializar diccionarios para almacenar predicciones y mejores hiperparámetros
    predictions = {name: [] for name in classifiers.keys()}
    best_params = {name: [] for name in classifiers.keys()}
    
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    y_true = []
    fold = 1
    for train_index, test_index in kfold.split(X, y_encoded):   
        X_train_cv, X_test_cv = X.iloc[train_index], X.iloc[test_index]
        y_train_cv, y_test_cv = y_encoded.iloc[train_index], y_encoded.iloc[test_index]
        y_true.append(y_test_cv.iloc[0])
        #y_true.append(y_test_cv[0])
        #imprime el numero de fold que se esta evaluando 
        print('Fold:', fold)
        
        # **Selección de Características Dentro del Bucle**
        scaler = StandardScaler()
        X_train_cv_scaled = scaler.fit_transform(X_train_cv)
        X_test_cv_scaled = scaler.transform(X_test_cv)

        X_train_final = X_train_cv_scaled
        X_test_final = X_test_cv_scaled
        
        # Iterar sobre los clasificadores
        for name, clf_info in classifiers.items():
            model = clf_info['model']
            params = clf_info['params']

            # Crear pipeline con el modelo
            pipeline = Pipeline([
                ('classifier', model)
            ])

            # Ajustar GridSearchCV
            grid = GridSearchCV(estimator=pipeline, param_grid=params, n_jobs=-1, cv=3)
            grid.fit(X_train_final, y_train_cv)
            y_pred = grid.predict(X_test_final)[0]
            predictions[name].append(y_pred)
            best_params[name].append(grid.best_params_)
        fold += 1
    
    for name in best_params.keys():
        print(f"Análisis de hiperparámetros para {name}:")
        # Combinar todos los parámetros en una lista de tuplas para contar frecuencias
        param_list = [tuple(sorted(params.items())) for params in best_params[name]]
        param_counts = Counter(param_list)
        for params, count in param_counts.items():
            print(f"  Hiperparámetros: {dict(params)} - Frecuencia: {count}")
        print("\n")

    train_full_model(X, y_encoded, best_params,classifiers)
    test_with_reserved(X_reserved, y_reserved,n_split)

In [13]:
def main():
    results_dir = 'resultados_peru_bio'
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
    # Cargar los splits desde el archivo pickle
    with open('random_splits_peru_bio.pkl', 'rb') as f:
        loaded_splits_peru = pickle.load(f)

    # Verificar los splits cargados
    #print(loaded_splits_peru)

    # Carga de datos
    DATA_PATH = '/data/sespinoza/objetivo2/model-iterativo/'
    data_peru = pd.read_csv(DATA_PATH + 'data_peru_prep_biomarkers.csv', header=0, index_col=0, sep=',', encoding='utf-8')
    data_peru = data_peru.reset_index(drop=True)

    X = data_peru.drop(columns=['labels'])
    y = data_peru['labels']
    le = LabelEncoder()
    y = le.fit_transform(y)
    y = pd.Series(y)
    for i, split in enumerate(loaded_splits_peru):
        X_train, X_test, y_train, y_test = split_data(X, y, split)
        print(f"Split {i+1}")
        print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")
        print(f"y_train shape: {y_train.shape}, y_test shape: {y_test.shape}")
        evaluate_with_splits(X_train, X_test, y_train, y_test,i)
        #print("Resultados guardados en resultados_peru.txt")

In [14]:
if __name__ == '__main__':
    main()

Split 1
X_train shape: (57, 37), X_test shape: (12, 37)
y_train shape: (57,), y_test shape: (12,)
Fold: 1


/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1

Fold: 2


/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/linea

Fold: 3


...

Fold: 4


/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1

Fold: 5


/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/sespinoza/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1

Análisis de hiperparámetros para SVM Linear:
  Hiperparámetros: {'classifier__C': 0.1} - Frecuencia: 4
  Hiperparámetros: {'classifier__C': 0.01} - Frecuencia: 1


Análisis de hiperparámetros para SVM RBF:
  Hiperparámetros: {'classifier__C': 1, 'classifier__gamma': 0.1} - Frecuencia: 5


Análisis de hiperparámetros para Logistic Regression (LASSO):
  Hiperparámetros: {'classifier__C': 1} - Frecuencia: 2
  Hiperparámetros: {'classifier__C': 0.1} - Frecuencia: 2
  Hiperparámetros: {'classifier__C': 10} - Frecuencia: 1


Análisis de hiperparámetros para Random Forest:
  Hiperparámetros: {'classifier__max_depth': None, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 100} - Frecuencia: 4
  Hiperparámetros: {'classifier__max_depth': None, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 200} - Frecuencia: 1


Análisis de hiperparámetros para Boosting:
  Hiperparámetros: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 100} - Frecuencia: 4
  Hipe